In [1]:
%matplotlib inline

In [2]:
cd /home/sathappan/workspace/time2event/src/

/home/sathappan/workspace/time2event/src


In [9]:
#from PointProcess.hawkes import UniVariateHawkes
import pandas as pd
import gzip
import numpy as np
from scipy.optimize import minimize
from datetime import timedelta
import tensorflow as tf
import json
from datetime import timedelta

In [11]:
with gzip.open("/Users/nikhil/phd/urban_computing/datasets/wmata/wmata_2015_2016/user_timeseries_sorted_100.gz" ) as inf:
    cnt = 0
    d2 = []
    import json
    from dateutil.parser import parse
    for i in inf:
        d2.append([parse(dt) for dt in json.loads(i.decode('utf-8'))['arrivalTimes']])
        cnt += 1
        if cnt > 100:
            break

In [12]:
dframe = pd.DataFrame(d2)

In [13]:
dframe.head()

,0,1,2,3,4,5,6,7,8,9,...,1530,1531,1532,1533,1534,1535,1536,1537,1538,1539
0,2015-01-03 18:58:04,2015-01-12 08:13:55,2015-01-13 08:20:31,2015-01-13 17:29:05,2015-01-14 08:16:17,2015-01-15 08:41:52,2015-01-15 09:21:01,2015-01-16 07:23:52,2015-01-16 11:27:28,2015-01-19 13:17:17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-01-05 08:44:57,2015-01-05 18:23:28,2015-01-06 08:48:43,2015-01-06 18:29:40,2015-01-07 10:04:20,2015-01-07 18:17:57,2015-01-08 08:47:50,2015-01-08 19:22:42,2015-01-09 08:47:20,2015-01-12 10:46:28,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-01-05 06:34:05,2015-01-05 17:02:29,2015-01-06 06:44:42,2015-01-06 17:04:40,2015-01-07 06:34:28,2015-01-07 17:02:43,2015-01-08 09:12:58,2015-01-08 17:02:58,2015-01-09 06:35:18,2015-01-09 17:02:37,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-01-05 06:51:04,2015-01-05 16:33:38,2015-01-07 06:51:18,2015-01-07 16:36:55,2015-01-08 06:56:46,2015-01-08 16:36:12,2015-01-09 06:47:21,2015-01-09 16:31:46,2015-01-12 08:57:45,2015-01-12 16:35:51,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-01-05 08:41:21,2015-01-05 18:25:38,2015-01-06 08:47:17,2015-01-06 17:32:12,2015-01-07 08:51:45,2015-01-07 18:04:10,2015-01-08 08:45:07,2015-01-08 18:01:10,2015-01-09 08:56:27,2015-01-09 18:28:55,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
dframe.columns = [str(l) for l in range(0, 1540)]
names = [str(l) for l in range(0, 1540)]
#df['newcol'] = df[['col1','col2']].sum(axis=1) - df['col5']
#df[['newcol', 'col3']].sub(df['col5'], axis=0)
arrivals = dframe[names[:400]].sub(dframe["0"], axis=0)
mask = ~(arrivals.isnull()).as_matrix()
arrivals_numpy = np.nan_to_num(arrivals.applymap(lambda x: x/np.timedelta64(1, '1D')).as_matrix())


In [7]:
def calc_Ai(arrivals, mask, beta):
    tdiff = np.diff(arrivals) * mask[:, 1:]
    tdiff = np.exp(tdiff * -beta)
    A_i = np.zeros_like(arrivals, dtype=np.float128)
    for i in range(tdiff.shape[1]):
        #print(i,)
        A_i[:, i + 1] = tdiff[:, i] * (1 + A_i[:, i])
    return A_i, tdiff

In [9]:
A_i, tempdiff = calc_Ai(arrivals_numpy, mask, 1.0)

#### Tensorflow fitting

In [100]:
def loglikelihood():
    #tdiff = diff(arrivals) * mask[:, 1:]
    #A_i = tf.zeros_like(arrivals)
    #for i in range(tdiff.shape[0]):
    #    A_i[:, i + 1] = tdiff[:, i] * (1 + A[:, i ])
    part1 = tf.reduce_sum(tf.log(mu + alpha * ais_tvar), axis=1)[:, None]
    T_max = tf.reduce_max(arrivals_tvar*mask_tvar, axis=1)[:, None]
    part2 = mu * T_max
    p3_tmp = tf.reduce_sum(tf.exp(-beta * ((T_max - arrivals_tvar)*mask_tvar)) - tf.constant(1.0, dtype=tf.float64), axis=1)
    part3 = (alpha / beta)* p3_tmp[:, None]
    
    #return part1, part2, part3, p3_tmp
    return -1*tf.reduce_sum(part1 - part2 + part3)

In [101]:
## Assuming number of users is 32
num_users = 101

## All parameters are column vectors with size equal to number of users

alpha = tf.Variable(tf.ones([num_users, 1], dtype=tf.float64), dtype=tf.float64)
beta = tf.Variable(tf.ones([num_users, 1], dtype=tf.float64), dtype=tf.float64)
mu = tf.Variable(tf.ones([num_users, 1], dtype=tf.float64), dtype=tf.float64)


In [102]:
T_max = 400
arrivals_tvar = tf.placeholder(tf.float64, [None, T_max])
mask_tvar = tf.placeholder(tf.float64, [None, T_max])
ais_tvar = tf.placeholder(tf.float64,[None, T_max])
tdiff_tvar = tf.placeholder(tf.float64, [None, T_max-1])
p3 = loglikelihood()
grad = tf.gradients(p3, [alpha, beta, mu])
optim = tf.train.GradientDescentOptimizer(0.01).minimize(p3)
#p1, p2, p3 = loglikelihood()

In [103]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    tf_a = (sess.run([p3, optim], feed_dict={arrivals_tvar: np.nan_to_num(arrivals_numpy), mask_tvar:mask, ais_tvar:A_i, tdiff_tvar: tempdiff}))
    #tf_a, tf_b, tf_c, tf_d = (sess.run(p3, feed_dict={arrivals_tvar: np.nan_to_num(arrivals_numpy), mask_tvar:mask, ais_tvar:A_i, tdiff_tvar: tempdiff}))

In [104]:
tf_a

[40826.057238954279, None]

In [68]:
tf_b.shape

(101, 1)

In [69]:

tf_d.shape

(101,)

In [70]:
tf_d.shape

(101,)

In [78]:
tf_a[14]

array([-57.18139879])

In [92]:

tf_a[0].sum()

-40826.057238954279